In [2]:
dbconnection = {
    "username": "neo4j",
    "password": "password",
    "url": "bolt://localhost:7687"
}
from neo4j import GraphDatabase
import json
from pathlib import Path
datapath = Path("../master-database-files/master-experimental/neo4j_citing_data_conversion/papers.json")
assert datapath.parent.exists()
with GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "password")) as driver:
    with driver.session() as session:
        result = session.run("MATCH (n:Paper) RETURN n, id(n) AS id")
        jsonarrayy = []
        for record in result:
            citedIds = [res["id"] for res in session.run("MATCH (n)-[CITES]->(m) WHERE id(n) = $id RETURN id(m) AS id", id=record["id"])]
            citingIds = [res["id"] for res in session.run("MATCH (n)<-[CITES]-(m) WHERE id(n) = $id RETURN id(m) AS id", id=record["id"])]
            jsonarrayy.append({
                "arxiv": record["n"]["arxiv"],
                "doi": record["n"]["doi"],
                "semscholar": record["n"]["semscholar"],
                "pdf": record["n"]["pdf"],
                "status": record["n"]["status"],
                "title": record["n"]["title"],
                "id": record["id"],
                "cited": citedIds,
                "citing": citingIds
            })
        with datapath.open("w") as f:
            json.dump(jsonarrayy, f, indent=2)